<a href="https://colab.research.google.com/github/AnhDung2k3/Stock_Market_Performance_Analysis_using_Python.ipynb/blob/main/Stock_Market_Performance_Analysis_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Stock Market Performance Analysis**

I start by importing the necessary Python libraries and the dataset. I use the Yahoo Finance API (yfinance) to collect the real-time stock market data for the past 3 months

Install the yfinance API in my Python environment:

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now, I will collect the real-time stock data using the yfinance API:

In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime

start_date = datetime.now() - pd.DateOffset(months = 3)
end_date = datetime.now()

tickers = ['AAPL', 'META', 'GOOG', 'MSFT']
#download the historical price data for 4 companies: AAPL(Apple Inc), META(Meta), GOOG(Google), MSFT(Microsoft Corp)

df_list = []

for ticker in tickers:
  data = yf.download(ticker, start = start_date, end = end_date)
  df_list.append(data)

df = pd.concat(df_list, keys = tickers, names = ['Ticker', 'Date'])
print(df.head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-03-20  155.070007  157.820007  154.149994  157.399994  157.182587   
       2023-03-21  157.320007  159.399994  156.539993  159.279999  159.059982   
       2023-03-22  159.300003  162.139999  157.809998  157.830002  157.612000   
       2023-03-23  158.830002  161.550003  157.679993  158.929993  158.710464   
       2023-03-24  158.860001  160.339996  157.850006  160.250000  160.028656   

                     Volume  
Ticker Date                  
AAPL   2023-03-20  73641400  
       2023-03-21  73938300  
       2023-03-22  75701800  
      

In this data set, Date column is the index column of this DataFrame. So, we need to reset the index

In [4]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-03-20  155.070007  157.820007  154.149994  157.399994   
1   AAPL 2023-03-21  157.320007  159.399994  156.539993  159.279999   
2   AAPL 2023-03-22  159.300003  162.139999  157.809998  157.830002   
3   AAPL 2023-03-23  158.830002  161.550003  157.679993  158.929993   
4   AAPL 2023-03-24  158.860001  160.339996  157.850006  160.250000   

    Adj Close    Volume  
0  157.182587  73641400  
1  159.059982  73938300  
2  157.612000  75701800  
3  158.710464  67622100  
4  160.028656  59196500  


The performance in the stock market of all the companies:

In [6]:
import plotly.express as px

fig = px.line(df, x = 'Date', y = 'Close', color = 'Ticker', title = 'Stock Market Performance for the last 3 months')
fig.show()

I will creat a faceted area chart to compare the performance of different companies and identify similarities or differences in their stock price movements easier

In [8]:
fig = px.area(df, x = 'Date', y = 'Close', color = 'Ticker',
              facet_col = 'Ticker',
              labels = {'Date': 'Date', 'Close': 'Closing Price', 'Ticker': 'Company'},
              title = 'Stock price for Apple, Meta, Google and Microsoft')
fig.show()

Now, I will analyze moving averages to identify trends and patterns in each company's stock price movements over a period of time:

In [9]:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window = 10).mean().reset_index(0, drop = True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window = 20).mean().reset_index(0, drop = True)

for ticker, group in df.groupby('Ticker'):
  print(f'Moving Averages for {ticker}')
  print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10        MA20
0          NaN         NaN
1          NaN         NaN
2          NaN         NaN
3          NaN         NaN
4          NaN         NaN
..         ...         ...
58  179.752002  176.529002
59  180.353001  177.091001
60  181.023001  177.685001
61  181.615001  178.351000
62  182.012001  178.844500

[63 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
126         NaN         NaN
127         NaN         NaN
128         NaN         NaN
129         NaN         NaN
130         NaN         NaN
..          ...         ...
184  124.498001  123.543000
185  124.477001  123.916500
186  124.578001  124.131001
187  124.720000  124.346500
188  124.603000  124.373500

[63 rows x 2 columns]
Moving Averages for META
           MA10        MA20
63          NaN         NaN
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
..          ...         ...
121  267.914996  257.54

Now, I visualize the moving averages:

In [10]:
for ticker, group in df.groupby('Ticker'):
  fig = px.line(group, x = 'Date', y = ['Close', 'MA10', 'MA20'],
                title = f'Moving Averages of {ticker}')
  fig.show()

From 4 separate graphs above, I realize that when the MA10 crosses above the MA20, the price of the stock will rise. And on the contrary, when the MA10 crosses below the MA20, the price of the stock will fall.

Right now, I will analyze the volatility - a measure of how much and how often the stock price or market fluctuates over a given period of time.

In [11]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window = 10).std().reset_index(0, drop = True)
fig = px.line(df, x = 'Date', y = 'Volatility', color = 'Ticker',
              title = 'Volatility of 4 companies')
fig.show()

High volatility indicates that the stock or market experiences large and frequent price movements, while low volatility indicates that the market experiences smaller or less frequent price movements.

The correlation between the stock price of Apple and Microsoft:

In [13]:
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns = {'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns = {'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on = 'Date')

fig = px.scatter(df_corr, x = 'AAPL', y = 'MSFT', trendline = 'ols', title = 'Correlation between Apple and Microsoft')
fig.show()

From the graph above, I can easily realize that there is a strong linear relationship between Apple and Microsoft. When the stock price of Apple increases, the stock price of Microsoft tends to raise also. This strong correlation could come from market conditions, industry trends or business partners. For investors, this positive sign will give them an opportunity by investing in both companies

I will create another graph about the correlation between Google and META:

In [14]:
google = df.loc[df['Ticker'] == 'GOOG', ['Date', 'Close']].rename(columns = {'Close': 'GOOG'})
meta = df.loc[df['Ticker'] == 'META', ['Date', 'Close']].rename(columns = {'Close': 'META'})
df_corre = pd.merge(google, meta, on = 'Date')

fig = px.scatter(df_corre, x = 'GOOG', y = 'META', trendline = 'ols', title = 'Correlation between Google and Meta')
fig.show()

This graph also show us a linear relation ship between Google and META. Although this line is not strong enough like the correlation between Apple and Microsoft, it is also a positive sign because when the stock value of Google increases, the stock value of META augments too.